In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
mkdir -p /content/drive/MyDrive/AI_TUTOR_QLORA/{data,adapter,deploy}


In [ ]:
BASE_DIR = "/content/drive/MyDrive/AI_TUTOR_QLORA"

DATA_DIR = f"{BASE_DIR}/data"
ADAPTER_DIR = f"{BASE_DIR}/adapter"
DEPLOY_DIR = f"{BASE_DIR}/deploy"

In [ ]:
!curl -L https://micro.mamba.pm/api/micromamba/linux-64/1.5.8 \
  | tar -xvj bin/micromamba


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  3947    0  3947    0     0   2067      0 --:--:--  0:00:01 --:--:--  2067
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0bin/micromamba
100 5394k  100 5394k    0     0  1551k      0  0:00:03  0:00:03 --:--:-- 3584k


In [ ]:
!curl -L https://micro.mamba.pm/api/micromamba/linux-64/1.5.8 \
  | tar -xvj bin/micromamba


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3943    0  3943    0     0   3588      0 --:--:--  0:00:01 --:--:--  3588
bin/micromamba
100 5394k  100 5394k    0     0  2291k      0  0:00:02  0:00:02 --:--:-- 7207k


In [ ]:
!file bin/micromamba


bin/micromamba: ELF 64-bit LSB pie executable, x86-64, version 1 (GNU/Linux), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, for GNU/Linux 2.6.18, stripped


In [ ]:
!./bin/micromamba create -y -n qlora -c conda-forge python=3.10


warning  libmamba 'root_prefix' set with default value: /root/micromamba
[+] 0.0s
[+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   6%
conda-forge/noarch     9%[+] 0.3s
conda-forge/linux-64  19%
conda-forge/noarch    37%[+] 0.4s
conda-forge/linux-64  24%
conda-forge/noarch    46%[+] 0.5s
conda-forge/linux-64  33%
conda-forge/noarch    64%[+] 0.6s
conda-forge/linux-64  41%
conda-forge/noarch    82%conda-forge/noarch                                
[+] 0.7s
conda-forge/linux-64  46%[+] 0.8s
conda-forge/linux-64  52%[+] 0.9s
conda-forge/linux-64  59%[+] 1.0s
conda-forge/linux-64  68%[+] 1.1s
conda-forge/linux-64  72%[+] 1.2s
conda-forge/linux-64  74%[+] 1.3s
conda-forge/linux-64  83%[+] 1.4s
conda-forge/linux-64  92%[+] 1.5s
conda-forge/linux-64  92%[+] 1.6s
conda-forge/linux-64  92%[+] 1.7s
conda-forge/linux-64  92%[+] 1.8s
conda-forge/linux-64  92%[+] 1.9s
conda-forge/linux-64  92%[+] 2.0s
conda-forge/linux-64  92%[+] 2.1s
conda-forge/linux-64  

In [ ]:
!./bin/micromamba run -n qlora python --version


warning  libmamba 'root_prefix' set with default value: /root/micromamba
Python 3.10.19


In [ ]:
!./bin/micromamba run -n qlora pip install -q \
  torch==2.2.2+cu118 \
  torchvision==0.17.2+cu118 \
  torchaudio==2.2.2+cu118 \
  --extra-index-url https://download.pytorch.org/whl/cu118
!./bin/micromamba run -n qlora pip install -q \
  transformers==4.36.2 \
  peft==0.7.1 \
  accelerate==0.25.0 \
  trl==0.7.10 \
  bitsandbytes==0.41.1 \
  datasets
!./bin/micromamba run -n qlora pip install bitsandbytes==0.41.1
!./bin/micromamba run -n qlora pip install -q "numpy<2"
!./bin/micromamba run -n qlora pip install -q scipy


warning  libmamba 'root_prefix' set with default value: /root/micromamba
warning  libmamba 'root_prefix' set with default value: /root/micromamba
warning  libmamba 'root_prefix' set with default value: /root/micromamba
warning  libmamba 'root_prefix' set with default value: /root/micromamba
warning  libmamba 'root_prefix' set with default value: /root/micromamba


In [ ]:
!./bin/micromamba run -n qlora python -c "import numpy, scipy, bitsandbytes; print('NumPy:', numpy.__version__); print('SciPy:', scipy.__version__); print('bitsandbytes:', bitsandbytes.__version__); print('OK')"
!./bin/micromamba run -n qlora python -c "import bitsandbytes; print('bitsandbytes import OK')"
!./bin/micromamba run -n qlora python -c "import importlib.metadata as m; print(m.version('bitsandbytes'))"


warning  libmamba 'root_prefix' set with default value: /root/micromamba
NumPy: 1.26.4
SciPy: 1.15.3
Traceback (most recent call last):
  File "<string>", line 1, in <module>
AttributeError: module 'bitsandbytes' has no attribute '__version__'
warning  libmamba 'root_prefix' set with default value: /root/micromamba
bitsandbytes import OK
warning  libmamba 'root_prefix' set with default value: /root/micromamba
0.41.1


In [ ]:
from datasets import load_dataset

# Stream the dataset (no full download)
dataset = load_dataset(
    "teknium/OpenHermes-2.5",
    split="train",
    streaming=True
)

# Take only first 800 samples
dataset_800 = dataset.take(800)

# Convert to normal Dataset object
from datasets import Dataset
dataset_800 = Dataset.from_list(list(dataset_800))

print(dataset_800)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Dataset({
    features: ['custom_instruction', 'topic', 'model_name', 'model', 'skip_prompt_formatting', 'category', 'conversations', 'views', 'language', 'id', 'title', 'idx', 'hash', 'avatarUrl', 'system_prompt', 'source'],
    num_rows: 800
})


In [ ]:
def format_chat(example):
    tutor_system = (
        "You are an AI tutor. "
        "Your job is to teach concepts clearly to beginners. "
        "Always explain answers step by step using simple language "
        "and examples when helpful."
    )

    convo = example["conversations"]
    text = tutor_system + "\n\n"

    for turn in convo:
        if turn["from"] == "human":
            text += f"User: {turn['value'].strip()}\n"

        elif turn["from"] == "gpt":
            text += "Assistant: Let’s understand this step by step.\n"
            text += f"{turn['value'].strip()}\n"

    return {"text": text.strip()}


In [ ]:
final_dataset = dataset_800.map(
    format_chat,
    remove_columns=dataset_800.column_names
)

print(final_dataset)


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 800
})


In [ ]:
for i in range(5):
    print(final_dataset[i]["text"])
    print("------")


You are an AI tutor. Your job is to teach concepts clearly to beginners. Always explain answers step by step using simple language and examples when helpful.

User: Every day, a tree drops 7 leaves. How many leaves would it drop in a month of February in a non-leap year? Include your logic.
Assistant: Let’s understand this step by step.
Here's the logic behind this:

1. We know that February has 28 days in a non-leap year.
2. If the tree drops 7 leaves every day, then over the course of February, it would drop:
   Leaves dropped in February = Leaves per day * Days in February
   = 7 leaves * 28 days
   = 196 leaves

So, the tree would drop 196 leaves in February in a non-leap year.
------
You are an AI tutor. Your job is to teach concepts clearly to beginners. Always explain answers step by step using simple language and examples when helpful.

User: In analytical chemistry, what is the principle behind the use of an internal standard in quantitative analysis?
A. It compensates for var

In [ ]:
final_dataset.save_to_disk(DATA_DIR)


Saving the dataset (0/1 shards):   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
!./bin/micromamba run -n qlora pip install -q sentencepiece
!./bin/micromamba run -n qlora pip install -q protobuf==3.20.3


warning  libmamba 'root_prefix' set with default value: /root/micromamba
warning  libmamba 'root_prefix' set with default value: /root/micromamba


In [ ]:
%%bash
cat << 'EOF' > load_mistral.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

print("Torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print("GPU:", torch.cuda.get_device_name(0))

model_name = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

BASE_DIR = "/content/drive/MyDrive/AI_TUTOR_QLORA/deploy"

tokenizer.save_pretrained(f"{BASE_DIR}/base_tokenizer")
model.config.save_pretrained(f"{BASE_DIR}/base_config")

print("💾 Tokenizer & base config saved to Google Drive")

print("✅ MISTRAL 7B LOADED SUCCESSFULLY IN 4-BIT")
EOF


In [ ]:
!./bin/micromamba run -n qlora python load_mistral.py


warning  libmamba 'root_prefix' set with default value: /root/micromamba
/root/micromamba/envs/qlora/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Torch: 2.2.2+cu118
CUDA: 11.8
GPU: Tesla T4
/root/micromamba/envs/qlora/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/micromamba/envs/qlora/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/micromamba/envs/qlora/lib/python3.10/site-packages/tr

In [ ]:
!./bin/micromamba run -n qlora pip install -q trl==0.7.10
!./bin/micromamba run -n qlora python -c "import trl; print('trl OK')"


warning  libmamba 'root_prefix' set with default value: /root/micromamba
warning  libmamba 'root_prefix' set with default value: /root/micromamba
/root/micromamba/envs/qlora/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/micromamba/envs/qlora/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/micromamba/envs/qlora/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
trl OK


In [ ]:
%%bash
cat << 'EOF' > train_qlora.py
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
BASE_DIR = "/content/drive/MyDrive/AI_TUTOR_QLORA"
DATA_DIR = f"{BASE_DIR}/data"
ADAPTER_DIR = f"{BASE_DIR}/adapter"

print("🚀 Starting QLoRA training")

# ------------------------
# Load dataset
# ------------------------
final_dataset = load_from_disk(DATA_DIR)


# ------------------------
# Load model (4-bit)
# ------------------------
model_name = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# 🔥 REQUIRED FOR QLoRA
model = prepare_model_for_kbit_training(model)

# Memory savings
model.gradient_checkpointing_enable()
model.config.use_cache = False

# ------------------------
# LoRA configuration
# ------------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# ------------------------
# Training arguments
# ------------------------
training_args = TrainingArguments(
    output_dir=ADAPTER_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    report_to="none",
)

# ------------------------
# Trainer
# ------------------------
trainer = SFTTrainer(
    model=model,
    train_dataset=final_dataset,
    peft_config=lora_config,
    tokenizer=tokenizer,
    args=training_args,
    formatting_func=lambda x: x["text"],
    max_seq_length=512,
)

trainer.train()
print("✅ QLoRA fine-tuning completed")
EOF


In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
!./bin/micromamba run -n qlora python train_qlora.py


warning  libmamba 'root_prefix' set with default value: /root/micromamba
/root/micromamba/envs/qlora/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/micromamba/envs/qlora/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
🚀 Starting QLoRA training
/root/micromamba/envs/qlora/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100% 2/2 [01:02<00:00, 31.26s/it]
/root/micromamba

In [ ]:
%%bash
cat << 'EOF' > save_adapter.py
from peft import PeftModel
from transformers import AutoModelForCausalLM

BASE_MODEL = "mistralai/Mistral-7B-v0.1"
ADAPTER_PATH = "/content/drive/MyDrive/AI_TUTOR_QLORA/adapter/checkpoint-200"
OUT_PATH = "/content/drive/MyDrive/AI_TUTOR_QLORA/deploy/adapter"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="cpu"
)

model = PeftModel.from_pretrained(model, ADAPTER_PATH)
model.save_pretrained(OUT_PATH)

print("✅ Final adapter saved for deployment")
EOF


In [ ]:
!./bin/micromamba run -n qlora python save_adapter.py


warning  libmamba 'root_prefix' set with default value: /root/micromamba
/root/micromamba/envs/qlora/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/micromamba/envs/qlora/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/micromamba/envs/qlora/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards:   0% 0/2 [00:00<?, ?it/s]